In [1]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import torch

In [2]:
from scipy.stats import uniform, norm
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [3]:
A = 1.2
b = -0.3
sigma = 0.2
N = 100

In [4]:
X = torch.from_numpy(uniform.rvs(size=N))
X.detach()
X = X.float()

In [5]:
e = torch.from_numpy(norm.rvs(loc=0, scale=sigma, size=N)).float()

In [6]:
y = A*X + b + e

In [7]:
y.detach()
y = y.float()

In [8]:
def sigma_to_rho(sigma):
    rho = torch.log(torch.exp(sigma) - 1)
    return rho

In [9]:
def rho_to_sigma(rho):
    sigma = torch.log(1 + torch.exp(rho))
    return sigma

In [10]:
def log_q_pdf(w, theta):
    (mu_A, mu_b, rho_A, rho_b) = theta 
    
    sigma_A = rho_to_sigma(rho_A)
    qA = torch.distributions.normal.Normal(mu_A, sigma_A)
    
    sigma_b = rho_to_sigma(rho_b)
    qb = torch.distributions.normal.Normal(mu_b, sigma_b)
    
    lp = qA.log_prob(w[0]) + qb.log_prob(w[1])
    return lp

In [11]:
def log_p_pdf(w):
    prior_mu_A = torch.tensor(0.0) 
    prior_sigma_A = torch.tensor(1.0)
    pA = torch.distributions.normal.Normal(prior_mu_A, prior_sigma_A)
    
    prior_mu_b = torch.tensor(0.0) 
    prior_sigma_b = torch.tensor(1.0)
    pb = torch.distributions.normal.Normal(prior_mu_b, prior_sigma_b)
    
    lp = pA.log_prob(w[0]) + pb.log_prob(w[1])
    return lp

In [12]:
def likelihood(y,X,w):
    A = w[0]
    b = w[1]
    
    y_pred = A*X + b
    lp = torch.distributions.normal.Normal(y_pred, 0.2).log_prob(y).sum()
    return lp

In [13]:
def f(w, theta):
    v = log_q_pdf(w, theta) - log_p_pdf(w) - likelihood(y,X,w)
    return v

In [14]:
mu_A = torch.tensor(0.0)
sigma_A = torch.tensor(0.1)
rho_A = sigma_to_rho(sigma_A)
mu_b = torch.tensor(0.0)
sigma_b = torch.tensor(0.1)
rho_b = sigma_to_rho(sigma_b)

theta = torch.tensor([mu_A, mu_b, rho_A, rho_b], requires_grad=True)

In [15]:

lst_grad_mu = []
lst_grad_rho = []
for j in range(0,5000):
    epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

    with torch.no_grad():
        w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
    w.requires_grad = True

    v = f(w, theta)   
    v.backward()
    v = torch.tensor([])

    with torch.no_grad():
        grad_mu = w.grad + theta.grad[0:2]
        grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


    lst_grad_mu.append(grad_mu)
    lst_grad_rho.append(grad_rho)

    theta.grad.zero_()
    w.grad.zero_()
    v.detach()
grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
        
 



In [16]:
grad_mu

tensor([-301.9706, -498.8301])

In [17]:
grad_rho

tensor([ 6.5926, 22.0693])

In [18]:
alpha = torch.tensor(10e-5)

In [19]:
for i in range(0,10000):
    lst_grad_mu = []
    lst_grad_rho = []
    for j in range(0,10):
        epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

        with torch.no_grad():
            w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
        w.requires_grad = True

        v = f(w, theta)   
        v.backward()
        v = torch.tensor([])

        with torch.no_grad():
            grad_mu = w.grad + theta.grad[0:2]
            grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


        lst_grad_mu.append(grad_mu)
        lst_grad_rho.append(grad_rho)

        theta.grad.zero_()
        w.grad.zero_()
        v.detach()
    grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
    grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
    
    #print(grad_mu)
    #print(grad_rho)
        
    with torch.no_grad():
        theta[0:2] -= alpha*grad_mu
        theta[2:4] -= alpha*grad_rho
    
    print(theta)



tensor([ 1.0310, -0.3495, -2.2532, -2.2538], requires_grad=True)
tensor([ 1.0409, -0.3362, -2.2528, -2.2547], requires_grad=True)
tensor([ 1.0588, -0.3098, -2.2544, -2.2567], requires_grad=True)
tensor([ 1.0756, -0.2842, -2.2546, -2.2594], requires_grad=True)
tensor([ 1.0757, -0.2904, -2.2552, -2.2618], requires_grad=True)
tensor([ 1.0844, -0.2795, -2.2560, -2.2654], requires_grad=True)
tensor([ 1.0925, -0.2700, -2.2560, -2.2673], requires_grad=True)
tensor([ 1.0928, -0.2756, -2.2570, -2.2710], requires_grad=True)
tensor([ 1.1031, -0.2636, -2.2579, -2.2724], requires_grad=True)
tensor([ 1.1142, -0.2497, -2.2586, -2.2740], requires_grad=True)
tensor([ 1.1235, -0.2389, -2.2599, -2.2777], requires_grad=True)
tensor([ 1.1267, -0.2390, -2.2603, -2.2800], requires_grad=True)
tensor([ 1.1232, -0.2494, -2.2603, -2.2831], requires_grad=True)
tensor([ 1.1264, -0.2500, -2.2607, -2.2841], requires_grad=True)
tensor([ 1.1252, -0.2583, -2.2620, -2.2859], requires_grad=True)
tensor([ 1.1244, -0.2644,

tensor([ 1.2379, -0.3234, -2.3375, -2.5041], requires_grad=True)
tensor([ 1.2377, -0.3239, -2.3380, -2.5048], requires_grad=True)
tensor([ 1.2391, -0.3219, -2.3386, -2.5053], requires_grad=True)
tensor([ 1.2432, -0.3150, -2.3390, -2.5061], requires_grad=True)
tensor([ 1.2489, -0.3038, -2.3394, -2.5069], requires_grad=True)
tensor([ 1.2443, -0.3119, -2.3404, -2.5073], requires_grad=True)
tensor([ 1.2538, -0.2956, -2.3413, -2.5086], requires_grad=True)
tensor([ 1.2485, -0.3068, -2.3419, -2.5111], requires_grad=True)
tensor([ 1.2569, -0.2933, -2.3432, -2.5134], requires_grad=True)
tensor([ 1.2443, -0.3168, -2.3446, -2.5159], requires_grad=True)
tensor([ 1.2423, -0.3221, -2.3451, -2.5167], requires_grad=True)
tensor([ 1.2455, -0.3153, -2.3455, -2.5178], requires_grad=True)
tensor([ 1.2431, -0.3189, -2.3464, -2.5189], requires_grad=True)
tensor([ 1.2398, -0.3257, -2.3466, -2.5202], requires_grad=True)
tensor([ 1.2382, -0.3295, -2.3473, -2.5230], requires_grad=True)
tensor([ 1.2510, -0.3082,

tensor([ 1.2499, -0.3245, -2.4106, -2.6618], requires_grad=True)
tensor([ 1.2474, -0.3300, -2.4114, -2.6635], requires_grad=True)
tensor([ 1.2475, -0.3297, -2.4117, -2.6644], requires_grad=True)
tensor([ 1.2502, -0.3246, -2.4123, -2.6652], requires_grad=True)
tensor([ 1.2461, -0.3322, -2.4126, -2.6661], requires_grad=True)
tensor([ 1.2501, -0.3237, -2.4124, -2.6679], requires_grad=True)
tensor([ 1.2469, -0.3306, -2.4130, -2.6686], requires_grad=True)
tensor([ 1.2413, -0.3399, -2.4137, -2.6701], requires_grad=True)
tensor([ 1.2394, -0.3436, -2.4142, -2.6708], requires_grad=True)
tensor([ 1.2442, -0.3355, -2.4150, -2.6727], requires_grad=True)
tensor([ 1.2422, -0.3404, -2.4153, -2.6739], requires_grad=True)
tensor([ 1.2446, -0.3356, -2.4159, -2.6741], requires_grad=True)
tensor([ 1.2442, -0.3369, -2.4162, -2.6746], requires_grad=True)
tensor([ 1.2469, -0.3315, -2.4167, -2.6761], requires_grad=True)
tensor([ 1.2493, -0.3266, -2.4174, -2.6771], requires_grad=True)
tensor([ 1.2462, -0.3334,

tensor([ 1.2521, -0.3276, -2.4724, -2.7928], requires_grad=True)
tensor([ 1.2581, -0.3174, -2.4729, -2.7930], requires_grad=True)
tensor([ 1.2585, -0.3161, -2.4732, -2.7936], requires_grad=True)
tensor([ 1.2627, -0.3089, -2.4741, -2.7942], requires_grad=True)
tensor([ 1.2573, -0.3185, -2.4745, -2.7946], requires_grad=True)
tensor([ 1.2544, -0.3227, -2.4751, -2.7956], requires_grad=True)
tensor([ 1.2548, -0.3215, -2.4756, -2.7964], requires_grad=True)
tensor([ 1.2589, -0.3150, -2.4759, -2.7970], requires_grad=True)
tensor([ 1.2575, -0.3173, -2.4763, -2.7974], requires_grad=True)
tensor([ 1.2512, -0.3284, -2.4768, -2.7981], requires_grad=True)
tensor([ 1.2459, -0.3378, -2.4776, -2.8000], requires_grad=True)
tensor([ 1.2459, -0.3392, -2.4787, -2.8015], requires_grad=True)
tensor([ 1.2459, -0.3392, -2.4791, -2.8022], requires_grad=True)
tensor([ 1.2551, -0.3227, -2.4796, -2.8033], requires_grad=True)
tensor([ 1.2485, -0.3348, -2.4801, -2.8041], requires_grad=True)
tensor([ 1.2527, -0.3264,

tensor([ 1.2594, -0.3173, -2.5287, -2.8865], requires_grad=True)
tensor([ 1.2569, -0.3217, -2.5290, -2.8869], requires_grad=True)
tensor([ 1.2588, -0.3192, -2.5293, -2.8871], requires_grad=True)
tensor([ 1.2517, -0.3324, -2.5298, -2.8881], requires_grad=True)
tensor([ 1.2561, -0.3237, -2.5301, -2.8886], requires_grad=True)
tensor([ 1.2525, -0.3288, -2.5302, -2.8892], requires_grad=True)
tensor([ 1.2525, -0.3279, -2.5304, -2.8903], requires_grad=True)
tensor([ 1.2583, -0.3173, -2.5308, -2.8911], requires_grad=True)
tensor([ 1.2616, -0.3108, -2.5313, -2.8921], requires_grad=True)
tensor([ 1.2557, -0.3219, -2.5315, -2.8929], requires_grad=True)
tensor([ 1.2503, -0.3316, -2.5320, -2.8939], requires_grad=True)
tensor([ 1.2487, -0.3352, -2.5325, -2.8954], requires_grad=True)
tensor([ 1.2506, -0.3311, -2.5327, -2.8960], requires_grad=True)
tensor([ 1.2506, -0.3299, -2.5330, -2.8969], requires_grad=True)
tensor([ 1.2536, -0.3237, -2.5334, -2.8974], requires_grad=True)
tensor([ 1.2519, -0.3266,

...

tensor([ 1.2568, -0.3213, -3.2036, -3.7517], requires_grad=True)
tensor([ 1.2557, -0.3231, -3.2035, -3.7518], requires_grad=True)
tensor([ 1.2540, -0.3251, -3.2035, -3.7517], requires_grad=True)
tensor([ 1.2551, -0.3231, -3.2036, -3.7517], requires_grad=True)
tensor([ 1.2560, -0.3217, -3.2036, -3.7518], requires_grad=True)
tensor([ 1.2542, -0.3253, -3.2037, -3.7519], requires_grad=True)
tensor([ 1.2540, -0.3256, -3.2037, -3.7519], requires_grad=True)
tensor([ 1.2531, -0.3274, -3.2038, -3.7520], requires_grad=True)
tensor([ 1.2529, -0.3277, -3.2039, -3.7521], requires_grad=True)
tensor([ 1.2533, -0.3269, -3.2039, -3.7521], requires_grad=True)
tensor([ 1.2526, -0.3278, -3.2039, -3.7521], requires_grad=True)
tensor([ 1.2525, -0.3272, -3.2039, -3.7521], requires_grad=True)
tensor([ 1.2536, -0.3253, -3.2039, -3.7522], requires_grad=True)
tensor([ 1.2533, -0.3256, -3.2040, -3.7522], requires_grad=True)
tensor([ 1.2539, -0.3244, -3.2041, -3.7523], requires_grad=True)
tensor([ 1.2544, -0.3235,

tensor([ 1.2538, -0.3257, -3.2082, -3.7560], requires_grad=True)
tensor([ 1.2555, -0.3233, -3.2081, -3.7559], requires_grad=True)
tensor([ 1.2548, -0.3242, -3.2081, -3.7560], requires_grad=True)
tensor([ 1.2548, -0.3246, -3.2082, -3.7560], requires_grad=True)
tensor([ 1.2552, -0.3234, -3.2082, -3.7560], requires_grad=True)
tensor([ 1.2560, -0.3220, -3.2082, -3.7561], requires_grad=True)
tensor([ 1.2559, -0.3217, -3.2083, -3.7561], requires_grad=True)
tensor([ 1.2554, -0.3226, -3.2084, -3.7562], requires_grad=True)
tensor([ 1.2551, -0.3233, -3.2083, -3.7562], requires_grad=True)
tensor([ 1.2526, -0.3276, -3.2084, -3.7563], requires_grad=True)
tensor([ 1.2554, -0.3229, -3.2085, -3.7564], requires_grad=True)
tensor([ 1.2562, -0.3216, -3.2084, -3.7564], requires_grad=True)
tensor([ 1.2532, -0.3264, -3.2085, -3.7564], requires_grad=True)
tensor([ 1.2513, -0.3295, -3.2085, -3.7565], requires_grad=True)
tensor([ 1.2521, -0.3282, -3.2085, -3.7564], requires_grad=True)
tensor([ 1.2545, -0.3244,

tensor([ 1.2557, -0.3242, -3.2124, -3.7603], requires_grad=True)
tensor([ 1.2554, -0.3253, -3.2124, -3.7603], requires_grad=True)
tensor([ 1.2550, -0.3253, -3.2123, -3.7603], requires_grad=True)
tensor([ 1.2549, -0.3253, -3.2123, -3.7602], requires_grad=True)
tensor([ 1.2547, -0.3262, -3.2124, -3.7602], requires_grad=True)
tensor([ 1.2551, -0.3254, -3.2124, -3.7603], requires_grad=True)
tensor([ 1.2546, -0.3262, -3.2125, -3.7603], requires_grad=True)
tensor([ 1.2554, -0.3246, -3.2124, -3.7603], requires_grad=True)
tensor([ 1.2572, -0.3211, -3.2125, -3.7603], requires_grad=True)
tensor([ 1.2551, -0.3249, -3.2126, -3.7605], requires_grad=True)
tensor([ 1.2559, -0.3232, -3.2126, -3.7605], requires_grad=True)
tensor([ 1.2552, -0.3247, -3.2127, -3.7605], requires_grad=True)
tensor([ 1.2573, -0.3215, -3.2127, -3.7606], requires_grad=True)
tensor([ 1.2561, -0.3233, -3.2126, -3.7606], requires_grad=True)
tensor([ 1.2552, -0.3243, -3.2127, -3.7606], requires_grad=True)
tensor([ 1.2547, -0.3249,

tensor([ 1.2553, -0.3245, -3.2164, -3.7651], requires_grad=True)
tensor([ 1.2546, -0.3260, -3.2164, -3.7652], requires_grad=True)
tensor([ 1.2551, -0.3252, -3.2164, -3.7652], requires_grad=True)
tensor([ 1.2549, -0.3253, -3.2164, -3.7652], requires_grad=True)
tensor([ 1.2573, -0.3216, -3.2165, -3.7652], requires_grad=True)
tensor([ 1.2580, -0.3199, -3.2166, -3.7652], requires_grad=True)
tensor([ 1.2572, -0.3220, -3.2166, -3.7652], requires_grad=True)
tensor([ 1.2555, -0.3252, -3.2166, -3.7652], requires_grad=True)
tensor([ 1.2566, -0.3231, -3.2166, -3.7652], requires_grad=True)
tensor([ 1.2560, -0.3236, -3.2166, -3.7652], requires_grad=True)
tensor([ 1.2546, -0.3264, -3.2167, -3.7653], requires_grad=True)
tensor([ 1.2555, -0.3245, -3.2167, -3.7652], requires_grad=True)
tensor([ 1.2552, -0.3251, -3.2167, -3.7652], requires_grad=True)
tensor([ 1.2558, -0.3238, -3.2168, -3.7652], requires_grad=True)
tensor([ 1.2537, -0.3273, -3.2168, -3.7653], requires_grad=True)
tensor([ 1.2517, -0.3307,

KeyboardInterrupt: 

In [170]:
rho_to_sigma(torch.tensor(0.5488))

tensor(1.0047)